In [1]:
# Imports
import torch
import torchvision # torch package for vision related things
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import (Dataset, DataLoader)  # Gives easier dataset managment by creating mini batches etc.
from tqdm import tqdm  # For nice progress bar!
import os
import pandas as pd
from PIL import Image

In [2]:
class PneumoniaDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = Image.open(img_path)
        image = image.convert('RGB')
        y_label = torch.tensor(int(self.annotations.iloc[index, 1]))

        if self.transform:
            image = self.transform(image)

        return (image, y_label)

In [3]:
# Simple CNN
class CNN(nn.Module):
    def __init__(self, in_channels=3, num_classes=2):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=8,
            kernel_size=(3, 3),
            stride=(1, 1),
            padding=(1, 1)
        )
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.conv2 = nn.Conv2d(
            in_channels=8,
            out_channels=16,
            kernel_size=(3, 3),
            stride=(1, 1),
            padding=(1, 1),
        )
        self.conv3 = nn.Conv2d(
            in_channels=16,
            out_channels=32,
            kernel_size=(3, 3),
            stride=(1, 1),
            padding=(1, 1),
        )
        self.avgpool = nn.AdaptiveAvgPool2d(7)
        self.fc1 = nn.Linear(32 * 7 * 7, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [4]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
IN_CHANNELS = 3
NUM_CLASSES = 2
LEARNING_RATE = 0.001
BATCH_SIZE = 32
N_EPOCHS = 5

In [5]:
transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Load Data
train_set = PneumoniaDataset(
    csv_file="chest_train_csv.csv",
    root_dir="chest_xray/train",
    transform=transforms
)

test_set = PneumoniaDataset(
    csv_file="chest_test_csv.csv",
    root_dir="chest_xray/test",
    transform=transforms
)

# create train and test loader
train_loader = DataLoader(dataset=train_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=BATCH_SIZE, shuffle=True)

In [6]:
# Initialize network
model = CNN(in_channels=IN_CHANNELS, num_classes=NUM_CLASSES).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [7]:
from torchsummary import summary

inputsize = (3, 224, 224)
print(summary(model, inputsize))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 8, 224, 224]             224
         MaxPool2d-2          [-1, 8, 112, 112]               0
            Conv2d-3         [-1, 16, 112, 112]           1,168
         MaxPool2d-4           [-1, 16, 56, 56]               0
            Conv2d-5           [-1, 32, 56, 56]           4,640
         MaxPool2d-6           [-1, 32, 28, 28]               0
 AdaptiveAvgPool2d-7             [-1, 32, 7, 7]               0
            Linear-8                  [-1, 512]         803,328
            Linear-9                    [-1, 2]           1,026
Total params: 810,386
Trainable params: 810,386
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.57
Forward/backward pass size (MB): 6.72
Params size (MB): 3.09
Estimated Total Size (MB): 10.38
------------------------------------------

In [10]:
# Train Network
for epoch in range(N_EPOCHS):
    losses = []

    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent
        optimizer.step()

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses)}")

100%|████████████████████████████████████████████████████████████████████████████████| 163/163 [01:50<00:00,  1.48it/s]


Cost at epoch 0 is 0.06468549609309965


100%|████████████████████████████████████████████████████████████████████████████████| 163/163 [02:27<00:00,  1.11it/s]


Cost at epoch 1 is 0.05387745819738752


100%|████████████████████████████████████████████████████████████████████████████████| 163/163 [01:27<00:00,  1.86it/s]


Cost at epoch 2 is 0.04573222670841254


100%|████████████████████████████████████████████████████████████████████████████████| 163/163 [01:23<00:00,  1.95it/s]


Cost at epoch 3 is 0.04044136191158344


100%|████████████████████████████████████████████████████████████████████████████████| 163/163 [01:25<00:00,  1.91it/s]

Cost at epoch 4 is 0.034360405733484624


In [11]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(f"Got {num_correct} / {num_samples} with accuracy \
            {float(num_correct)/float(num_samples)*100:.2f}")

    model.train()


print("Checking accuracy on Training Set")
check_accuracy(train_loader, model)

print("Checking accuracy on Test Set")
check_accuracy(test_loader, model)

Checking accuracy on Training Set
Got 5086 / 5216 with accuracy             97.51
Checking accuracy on Test Set
Got 427 / 624 with accuracy             68.43
